In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision.models import resnet18

class ImageSimilarityModel(nn.Module):
    def __init__(self):
        super(ImageSimilarityModel, self).__init__()
        self.resnet = resnet18(pretrained=True)
        for param in self.resnet.parameters():
            param.requires_grad = False
        self.mha = nn.MultiheadAttention(embed_dim=1000, num_heads=8)
        self.fc1=nn.Sequential(nn.Linear(1000,1),nn.Sigmoid())

    def forward(self, input1, input2):
        # Pass the inputs through the pre-trained ResNet18 model to get 512-dimensional embeddings
        query = self.resnet(input1)
        key = self.resnet(input2)
        value = self.resnet(input2)

        # Pass the query, key, and value through the Multi-Head Attention layer to get the similarity score
        attn_output, attn_output_weights = self.mha(query, key, value)

        # The output of the Multi-Head Attention layer is a tuple of the attention output and attention weights
        # We only need the attention output, which is a tensor of shape (batch_size, 1, 512)
        # We use the squeeze() method to remove the extra dimension of size 1
        attn_output=self.fc1(attn_output)

        similarity_score = attn_output.squeeze(1)
        return similarity_score

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchmetrics.functional.pairwise import pairwise_euclidean_distance
import torchmetrics
import lightning as l
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from PIL import Image
import pandas as pd
class CustomDataset(Dataset):
    def __init__(self, csv_file, transform=None):
        self.data = pd.read_csv(csv_file)[:4000]
        self.transform = transform

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        img_name1 = self.data.iloc[idx, 0]
        image1 = Image.open(img_name1).convert("RGB")

        img_name2 = self.data.iloc[idx, 1]
        image2 = Image.open(img_name2).convert("RGB")

        
        if self.transform:
            image1 = self.transform(image1)
            image2 = self.transform(image2)
        
        #create a image2 tensor same as the size of image2 with 1 as value
        image_like1 = torch.ones_like(image1)
        image_like2 = torch.ones_like(image2)

        label = int(self.data.iloc[idx, -1])

        return image_like1-image1,image_like2-image2, label

class CustomDataModule1(l.LightningDataModule):
    def __init__(self,  transform=None, batch_size=64):
        super().__init__()
        self.transform = transform
        self.batch_size = batch_size

    def setup(self, stage=None):
        self.train_dataset = CustomDataset("data/CEDAR_train.csv", transform=self.transform)
        self.val_dataset = CustomDataset("data/CEDAR_val.csv", transform=self.transform)
        self.test_dataset = CustomDataset("data/CEDAR_test.csv", transform=self.transform)
        

    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True,num_workers=7)

    def val_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=self.batch_size, shuffle=False,num_workers=7)
    
    def test_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=self.batch_size, shuffle=False,num_workers=7)

class LightningModel1(l.LightningModule):
    def __init__(self, model, learning_rate):
        super().__init__()

        self.learning_rate = learning_rate
        self.model = model

        # Save settings and hyperparameters to the log directory
        # but skip the model parameters
        self.save_hyperparameters(ignore=['model'])

        self.train_acc = torchmetrics.Accuracy(task="binary")
        self.val_acc = torchmetrics.Accuracy(task="binary")
        # self.val_auc=torchmetrics.ROC(task="binary")
        self.val_recall=torchmetrics.Recall(task="binary")
        self.val_precision=torchmetrics.Precision(task="binary")
        # self.test_auc=torchmetrics.ROC(task="binary")
        self.test_acc = torchmetrics.Accuracy(task="binary")
        self.test_recall=torchmetrics.Recall(task="binary")
        self.test_precision=torchmetrics.Precision(task="binary")

    def forward(self, x,y):
        return self.model(x,y)

    def _shared_step(self, batch):
        input,original, true_labels = batch
        logits= self(input,original)
        loss = F.binary_cross_entropy(logits, true_labels.float())
        predicted_labels = (logits > 0.5).float()
        # print(true_labels.shape,predicted_labels.shape)
        return loss, true_labels, predicted_labels

    def training_step(self, batch, batch_idx):
        loss, true_labels, predicted_labels = self._shared_step(batch)

        self.log("train_loss", loss)
        self.train_acc(predicted_labels, true_labels)
        self.log(
            "train_acc", self.train_acc, prog_bar=True, on_epoch=True, on_step= True
        )
        return loss

    def validation_step(self, batch, batch_idx):
        loss, true_labels, predicted_labels = self._shared_step(batch)

        self.log("val_loss", loss, prog_bar=True,on_step=True)
        self.val_acc(predicted_labels, true_labels)
        self.log("val_acc", self.val_acc, prog_bar=True,on_step=True)
        self.val_recall(predicted_labels, true_labels)
        self.log("val_recall", self.val_recall, prog_bar=True,on_step=True)
        self.val_precision(predicted_labels, true_labels)
        self.log("val_precision", self.val_precision, prog_bar=True,on_step=True)

    def test_step(self, batch, batch_idx):
        _, true_labels, predicted_labels = self._shared_step(batch)
        self.test_acc(predicted_labels, true_labels)
        self.log("test_acc", self.test_acc)
        self.test_recall(predicted_labels, true_labels)
        self.log("test_recall", self.test_recall)
        self.test_precision(predicted_labels, true_labels)
        self.log("test_precision", self.test_precision)
    

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        return optimizer

In [6]:
import lightning as l
import torch
from lightning.pytorch.callbacks import ModelCheckpoint
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
from lightning.pytorch.loggers import CSVLogger
from torchvision import transforms

torch.manual_seed(123)

transform = transforms.Compose([
transforms.Resize((256, 256)),
transforms.ToTensor(),
transforms.Lambda(lambda img: img / 255.0)])

dm = CustomDataModule1(batch_size=64,transform=transform)

pytorch_model = ImageSimilarityModel()

lightning_model = LightningModel1(model=pytorch_model, learning_rate=0.00005)


checkpoint_callback = ModelCheckpoint(
    dirpath="./models",filename="best_model1",
    save_top_k=1,monitor="val_loss", mode="min"
)
early_stopping_callback = EarlyStopping(
    monitor="val_acc", patience=3, verbose=True, mode="max"
)

trainer = l.Trainer(callbacks=[checkpoint_callback,early_stopping_callback],
    max_epochs=6,
    logger=CSVLogger(save_dir="logs/", name="model1_logs"),
    accelerator="cpu",
    devices="auto",
)

trainer.fit(model=lightning_model, datamodule=dm)


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
c:\Users\ASUS\Desktop\signature verification similarity\.venv\Lib\site-packages\lightning\pytorch\callbacks\model_checkpoint.py:653: Checkpoint directory C:\Users\ASUS\Desktop\signature verification similarity\models exists and is not empty.

  | Name           | Type                 | Params
--------------------------------------------------------
0 | model          | ImageSimilarityModel | 15.7 M
1 | train_acc      | BinaryAccuracy       | 0     
2 | val_acc        | BinaryAccuracy       | 0     
3 | val_recall     | BinaryRecall         | 0     
4 | val_precision  | BinaryPrecision      | 0     
5 | test_acc       | BinaryAccuracy       | 0     
6 | test_recall    | BinaryRecall         | 0     
7 | test_precision | BinaryPrecision      | 0     
--------------------------------------------------------
4.0 M     Trainable params
11.7 M    

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

c:\Users\ASUS\Desktop\signature verification similarity\.venv\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:436: Consider setting `persistent_workers=True` in 'val_dataloader' to speed up the dataloader worker initialization.
